In [1]:
import re
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from sklearn import preprocessing
from collections import Counter
import numpy as np
from sklearn.manifold import TSNE
import json
import matplotlib.pyplot as plt
from scipy.stats import norm, kurtosis
##中文標註
from scipy.stats import entropy
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False

In [2]:
ws = WS("./data", disable_cuda=False)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
pos = POS("./data", disable_cuda=False)

In [4]:
ner = NER("./data", disable_cuda=False)

In [5]:
def posArrangement(pos_hiddenState, index_list, s_l):
    hiddenState=[]
    for i in range(2):
        pos_hiddenState_all=[]
        pos_hidden = [None]*pos_hiddenState[i].shape[0]
        for partial_index, index in enumerate(index_list):
            pos_hidden[index] = pos_hiddenState[0][partial_index][:s_l[partial_index]].tolist()
            pos_hiddenState_all += pos_hidden[index]
        hiddenState.append(np.array(pos_hiddenState_all))
    return hiddenState

In [6]:
def newOrder_entity_set(entity_sentence_list,value):
    entity_set = set()
    for key in entity_sentence_list:
        entity_set.add((key[0]+value,key[1]+value,key[2],key[3]))
    return entity_set

In [7]:
def each_char(seq_sentence_list, pos_sentence_list,logits,label_sentence_list,sentence_list,entity_set,predict_avg_set,label_set,ws_count,pos_,char,charIndex,content):
    print(len(seq_sentence_list))
    kur_threshold = 50
    result = np.where(logits<0, 0, logits)
    #     print(len(pos_sentence_list))
    #different form predict label
    entity_list_order = ["O"]*len(sentence_list)
    for entity in sorted(entity_set):
        for k in range(entity[0],entity[1]):
            entity_list_order[k]=entity[2]
    
    char_list=[]
    entropyList=[]
    for k,i in enumerate(label_sentence_list):          
            
        label = label_sentence_list[k].split(':')
        if (len(label)<2):
            label.append("non")

        sort = np.argsort(-logits[k]) #機率大到小
        val = logits[k][sort]
        diff = abs(val[0]-val[1])
        entropy_ = entropy(result[k])
        entropyList.append(entropy_)

        char_list.append({"character" : sentence_list[k],
                        "index" : str(k),
                        "predicts" : predict_avg_set[k],
                        "label_list" : label_set[k],
                        "entropy" : str(np.round(entropy_,2)),
                        "ner_nonCollect" : (label[0],label[1]),
                        "ner" : entity_list_order[k],
                        "BI" : ws_count[k],
                        "pos" : pos_[k],
                        "index": {'char':str(char[k]+charIndex[k]),'article':str(content[k])}
                         })

    return char_list,ws_count,entropyList

In [8]:
def hiddenSplit(hiddenState_all,pos,ws):
    ner_h = hiddenState_all[0][0]
    ner_h1 =hiddenState_all[1][0]
    
    pos_h = pos[0]
    pos_h1 =pos[1]
    
    ws_h = ws[0][0]
    ws_h1 =ws[1][0]
    
    return ner_h, 


ner_h1, pos_h, pos_h1, ws_h, ws_h1

In [9]:
section=['台北市政府下午召開記者會，對於中央公布數字指出，截至今早7時，北市專責病床共1159個']
word_sentence_list, seq_sentence_list,ws_logits,ws_hiddenState_all = ws(section)
pos_sentence_list,pos_logits,pos_hiddenState,_, s_l, _,index_list = pos(word_sentence_list)

In [10]:
pos_hiddenState_all = posArrangement(pos_hiddenState, index_list, s_l)

In [11]:
entity_sentence_list, label_sentence_list, logits, hiddenState_all,c_v,w_v,label_list = ner(word_sentence_list, pos_sentence_list)

In [12]:
print("test OK, below to make json")

test OK, below to make json


In [13]:
f = open("news.txt",'r',encoding="utf-8")
onto=[]
onto_sect=[]
onto_string=""
for line in f.readlines():
    line = line.replace("\n","")
    line = line.replace(" ","")
    line = line.replace("纔","才")
    onto_string += line
    onto_sect.append([line])
onto.append(onto_string)
onto_list=list(onto[0])
onto_sort = np.array(onto_list)

In [14]:
ner_hidden=np.empty([0,1000])
ner_hidden1=np.empty([0,1000])
pos_hidden=np.empty([0,1000])
pos_hidden1=np.empty([0,1000])
ws_hidden=np.empty([0,1000])
ws_hidden1=np.empty([0,1000])

tra_word=[]
tra_ws=[]
tra_pos=[]
tra_label=[]
tra_logits=np.empty([0,73])
tra_entity=set()
section_len=0
charBegin = []
charIndex=[]
content =[]
for i,section in enumerate(onto_sect[:]):
#     print("\n")
#     print(i,section,len(section[0]))
    word_sentence_list, seq_sentence_list,ws_logits,ws_hiddenState_all = ws(section)
    pos_sentence_list,pos_logits,pos_hiddenState,_, s_l, _,index_list = pos(word_sentence_list)
    #
    pos_hiddenState_all = posArrangement(pos_hiddenState, index_list, s_l)
    entity_sentence_list, label_sentence_list, logits, hiddenState_all,c_v,w_v,label_list = ner(word_sentence_list, pos_sentence_list)
    #
    ner_h, ner_h1, pos_h, pos_h1, ws_h, ws_h1 = hiddenSplit(hiddenState_all, pos_hiddenState_all, ws_hiddenState_all)
    ner_hidden = np.concatenate((ner_hidden,ner_h), axis=0)
    ner_hidden1 = np.concatenate((ner_hidden1,ner_h1), axis=0)
    pos_hidden = np.concatenate((pos_hidden,pos_h), axis=0)
    pos_hidden1 = np.concatenate((pos_hidden1,pos_h1), axis=0)
    ws_hidden = np.concatenate((ws_hidden,ws_h), axis=0)
    ws_hidden1 = np.concatenate((ws_hidden1,ws_h1), axis=0)
    #
    tra_word += word_sentence_list[0]
    tra_ws += seq_sentence_list
    tra_pos += pos_sentence_list[0]
    tra_label += label_sentence_list
    tra_logits = np.concatenate((tra_logits,logits))
    #
    entity_set = newOrder_entity_set(entity_sentence_list,section_len)
    tra_entity = tra_entity.union(entity_set)
    charBegin += ([section_len]*len(section[0]))
    charIndex += list(np.arange(0,len(section[0])))
    content += [i]*len(section[0])
    section_len = section_len + len(section[0])
    print(i,len(tra_logits),len(charIndex))

0 756 756
1 917 917
2 1005 1005
3 1207 1207
4 1312 1312
5 1822 1822
6 2046 2046
7 2121 2121
8 2304 2304
9 2411 2411
10 2551 2551
11 2584 2584
12 2696 2696
13 2707 2707
14 2828 2828
15 2929 2929
16 2967 2967
17 2976 2976
18 3114 3114
19 3198 3198
20 3285 3285
21 3400 3400
22 3501 3501
23 3629 3629
24 3637 3637
25 3706 3706
26 3801 3801
27 3877 3877
28 4208 4208
29 4327 4327
30 4457 4457
31 4545 4545
32 4664 4664
33 4824 4824
34 4990 4990
35 5043 5043
36 5160 5160
37 5271 5271
38 5595 5595
39 5683 5683
40 5796 5796
41 5853 5853
42 5969 5969
43 6580 6580
44 6679 6679
45 6800 6800
46 6942 6942
47 7084 7084
48 7130 7130
49 7337 7337
50 7408 7408
51 7564 7564
52 7625 7625
53 7688 7688
54 7847 7847
55 7943 7943
56 8087 8087
57 8155 8155
58 8246 8246
59 8436 8436
60 8543 8543
61 8657 8657
62 8745 8745
63 8812 8812
64 8899 8899
65 9083 9083
66 9392 9392
67 9543 9543
68 9657 9657
69 9742 9742
70 9896 9896
71 9957 9957
72 10068 10068
73 10142 10142
74 10214 10214
75 10314 10314
76 10453 10453
77 

In [15]:
label_list_ = np.array(label_list).copy()
for i,label in enumerate(label_list):
    label_list_[i] = label.split(':')[0]
label_list = np.array(label_list)
label_list[:5]

array(['O', 'GPE:S', 'GPE:B', 'GPE:I', 'GPE:E'], dtype='<U13')

In [16]:
onto_sect[92]

['雲林縣政府於昨天下午13時46分接獲CDC通知，COVID-19確診案1129案於4月25日曾涉足雲林北港鎮，縣府立即啟動防疫機制。']

In [17]:
predict_avg_set = []
label_set = []
result = np.where(tra_logits<0, 0, tra_logits)
for i, char in enumerate(result):
    predict_index = np.argsort(-char)
    predict_value = char[predict_index]
    predict_sum = np.sum(predict_value)
    predict_avg = (predict_value/predict_sum)[:3].tolist()
    label_ = label_list[predict_index][:3].tolist()
    predict_avg_set.append(predict_avg)
    label_set.append(label_)

In [18]:
pos_=[]
index2word=[]
temp_pos = 0
for k,i in enumerate(tra_ws):
    if (i == 'B'):
        pos_c = tra_pos[temp_pos]
        pos_.append(pos_c)
        temp_pos +=1
    else:
        pos_.append(pos_c)
    index2word.append(temp_pos-1)

In [19]:
tra_ws.append('B')
ws_count = []
for k,i in enumerate(tra_ws):
    if (i == 'B'):
        count =1 
        for j in range(k+1,len(tra_ws)):
            if(tra_ws[j] =='I'):
                count += 1
            else:
                ws_count.append(count)
                zero_ = list(np.zeros(count-1))
                ws_count = ws_count+zero_
                break

In [20]:
char_list,word_count,entropyList = each_char(tra_ws,tra_pos,tra_logits,tra_label,onto[0],tra_entity,predict_avg_set,label_set,ws_count,pos_,charBegin,charIndex,content)

15745


In [21]:
char_list

[{'character': '關',
  'index': {'char': '0', 'article': '0'},
  'predicts': [1.0, 0.0, 0.0],
  'label_list': ['O', 'WORK_OF_ART:I', 'WORK_OF_ART:B'],
  'entropy': '0.0',
  'ner_nonCollect': ('O', 'non'),
  'ner': 'O',
  'BI': 2,
  'pos': 'P'},
 {'character': '於',
  'index': {'char': '1', 'article': '0'},
  'predicts': [1.0, 0.0, 0.0],
  'label_list': ['O', 'WORK_OF_ART:I', 'WORK_OF_ART:B'],
  'entropy': '0.0',
  'ner_nonCollect': ('O', 'non'),
  'ner': 'O',
  'BI': 0.0,
  'pos': 'P'},
 {'character': '「',
  'index': {'char': '2', 'article': '0'},
  'predicts': [1.0, 0.0, 0.0],
  'label_list': ['O', 'WORK_OF_ART:I', 'WORK_OF_ART:B'],
  'entropy': '0.0',
  'ner_nonCollect': ('O', 'non'),
  'ner': 'O',
  'BI': 1,
  'pos': 'PARENTHESISCATEGORY'},
 {'character': '台',
  'index': {'char': '3', 'article': '0'},
  'predicts': [0.5972913666730469, 0.34912787243334786, 0.05358076089360526],
  'label_list': ['O', 'GPE:B', 'PERSON:B'],
  'entropy': '0.83',
  'ner_nonCollect': ('O', 'non'),
  'ner': 

In [22]:
with open('Characters.json', 'w') as outfile:
    json.dump(char_list, outfile)

In [23]:
hiddenState = [None]*6 

In [24]:
hiddenState[0] = ner_hidden
hiddenState[1] = ner_hidden1
hiddenState[2] = pos_hidden
hiddenState[3] = pos_hidden1
hiddenState[4] = ws_hidden
hiddenState[5] = ws_hidden1

In [25]:
np.save('Hiddenstates.npy', hiddenState)

### for overview

In [26]:
len(tra_word),len(tra_entity),len(entropyList)

(9401, 1066, 15744)

In [27]:
wordEntity = ["O"]*len(tra_word)
for entity in sorted(tra_entity):
    wordIndexStart = index2word[entity[0]]
    wordIndexEnd = index2word[entity[1]]
    wordLength = wordIndexEnd - wordIndexStart
    wordEntity[wordIndexStart:wordIndexEnd] = [entity[2]] * wordLength

In [28]:
word2index=[0]*(len(tra_word))
for index in range(1,len(tra_word)):
    word2index[index] = len(tra_word[index-1])+word2index[index-1]

In [118]:
np.save('word2index.npy', word2index)

In [29]:
def maxEntropyWord(charList):
    entropyList=[]
    for i in charList:
        entropyList.append(i['entropy'])
    
    maxEntropy = max(entropyList)
    return maxEntropy

In [30]:
wordCollect=[]
entropyOrder=[]
entropyMax=0
entropyMin=1000
for index, word  in enumerate(tra_word):
    eachWord={}
    eachWord['word'] = word
    eachWord['ner'] = wordEntity[index]
    start = word2index[index]
    eachWord['dount'] = char_list[start:start+len(word)]
    maxWordEntropy = maxEntropyWord(char_list[start:start+len(word)])
    eachWord['max_entropy'] = float(maxWordEntropy)   
    entropyOrder.append(float(maxWordEntropy))

    #if it don't order
#     if (maxWordEntropy > entropyMax):
#         entropyMax = maxWordEntropy
#     if ( maxWordEntropy > 0 and  maxWordEntropy < entropyMin):
#         entropyMin = maxWordEntropy
#     entropyMax = np.round(entropyMax,2)
#     entropyMin = np.round(entropyMin,2)
#     entro = {"max":str(entropyMax),"min":str(entropyMin)}
        
    wordCollect.append(eachWord)

In [80]:
overviewRange = 2000
entropyOrder = np.array(entropyOrder)
entropySort = np.argsort(-entropyOrder)[:overviewRange]

In [81]:
wordCollect_ = np.array(wordCollect)[entropySort]
wordCollect_ = list(wordCollect_)

In [108]:
wordRecode=[]
for index, word in enumerate(wordCollect_):
    wordRecode.append(word['word'])

In [83]:
entropyMax = entropyOrder[entropySort][0]
entropyMin = entropyOrder[entropySort][overviewRange-1]

In [84]:
entropyMax,entropyMin

(2.48, 0.52)

In [85]:
entro = {"max":entropyMax,"min":entropyMin}

In [120]:
uncertainty={'word_collect':wordCollect_,'range':entro,'label': list(set(label_list_)),'wordRecode':tra_word}

In [111]:
uncertainty['label']

['DATE',
 'LANGUAGE',
 'MONEY',
 'LOC',
 'O',
 'LAW',
 'PERSON',
 'NORP',
 'FAC',
 'EVENT',
 'WORK_OF_ART',
 'QUANTITY',
 'PRODUCT',
 'ORG',
 'CARDINAL',
 'ORDINAL',
 'GPE',
 'PERCENT',
 'TIME']

In [121]:
with open('uncertainty.json', 'w') as outfile:
    json.dump(uncertainty, outfile)

In [140]:
thesame=[]
for index,w in enumerate(tra_word):
    if (w == "中國"):
        thesame.append(index)
        print(w)
        
wordIndex = np.array(tra_word)[thesame]

中國
中國
中國
中國
中國
中國
中國
中國
中國
中國
中國
中國
中國
中國
中國
中國
中國
中國


In [142]:
for i in thesame:
    print(i,np.array(word2index)[i])


476 798
1393 2387
1631 2805
1862 3203
1875 3228
2199 3777
2230 3831
2262 3897
6079 10254
8888 14873
8932 14949
8959 14993
8982 15033
9003 15069
9057 15162
9084 15213
9169 15353
9341 15643


In [138]:
char_list

[{'character': '此',
  'entropy': '0.0',
  'ner': 'O',
  'index': {'char': 15677, 'article': 113}},
 {'character': '外',
  'entropy': '0.0',
  'ner': 'O',
  'index': {'char': 15678, 'article': 113}},
 {'character': '，',
  'entropy': '0.0',
  'ner': 'O',
  'index': {'char': 15679, 'article': 113}},
 {'character': '戴',
  'entropy': '1.12',
  'ner': 'PERSON',
  'index': {'char': 15680, 'article': 113}},
 {'character': '琪',
  'entropy': '1.01',
  'ner': 'PERSON',
  'index': {'char': 15681, 'article': 113}},
 {'character': '表',
  'entropy': '0.0',
  'ner': 'O',
  'index': {'char': 15682, 'article': 113}},
 {'character': '示',
  'entropy': '0.0',
  'ner': 'O',
  'index': {'char': 15683, 'article': 113}},
 {'character': '，',
  'entropy': '0.0',
  'ner': 'O',
  'index': {'char': 15684, 'article': 113}},
 {'character': '撰',
  'entropy': '0.0',
  'ner': 'O',
  'index': {'char': 15685, 'article': 113}},
 {'character': '寫',
  'entropy': '0.0',
  'ner': 'O',
  'index': {'char': 15686, 'article': 113}}

### for content view

In [37]:
def each_char_temp(seq_sentence_list, pos_sentence_list,logits,sentence_list,entity_set,sum_length,i_):
    entropyMax=0
    entropyMin=10
    entity_list_order = ["O"]*len(sentence_list)
    for entity in sorted(entity_set):
        for k in range(entity[0],entity[1]):
            entity_list_order[k]=entity[2]    

    pie = dict()
    result = np.where(logits<0, 0, logits)
    pos_char_list=[]
    char_list=[]
    ws_count = []
    temp=-1
    entropySum=0
    for k,ws in enumerate(seq_sentence_list):
        if ws == 'B':
            temp+=1
            pos = pos_sentence_list[temp] #update pos
            pos_char_list.append(pos)
            count =1 
            for j in range(k+1,len(seq_sentence_list)):
                if(seq_sentence_list[j] =='I'):
                    count += 1
                else:
                    ws_count.append(count)
                    zero_ = list(np.zeros(count-1))
                    ws_count = ws_count+zero_
                    break
        else:
            pos_char_list.append(pos)
        label = label_sentence_list[k].split(':')
        if (len(label)<2):
            label.append("non")
        
        sort = np.argsort(-logits[k]) #機率大到小
        val = logits[k][sort]
        diff = abs(val[0]-val[1])
        entropy_ = entropy(result[k])
        entropySum += entropy_
        print(sum_length,k)
        char_list.append({"character" : sentence_list[k],
#                         "index" : str(k),
#                         "predicts" : predict_avg_set[k],
#                         "label_list" : label_set[k],
                        "entropy" : str(np.round(entropy_,2)),
#                         "ner_nonCollect" : (label[0],label[1]),
                        "ner" : entity_list_order[k],
                        "index": {'char':sum_length+k,'article':i_}
                          
#                         "BI" : ws_count[k],
#                         "pos" : pos_sentence_list[temp]
                         })
        if (entropy_ > entropyMax):
            entropyMax = entropy_
        if ( entropy_ > 0 and entropy_ < entropyMin):
            entropyMin = entropy_
        entropyAverage= entropySum/len(sentence_list)
        
        
        temp_entity = str(entity_list_order[k])
        
        if (temp_entity in pie):
            pie[temp_entity] +=1
        else:
            pie[temp_entity] =1

            
    print(pie)
    
    return char_list,entropyMax,entropyMin,entropyAverage,pie,len(sentence_list)

In [38]:
def legenSet(char_list):
    legen=set()
    for i in char_list:
        legen.add(i['ner'])
    return list(legen)

In [39]:
temp=[]
entropyMax=0
entropyMin=10
temp_sum_length=0
sum_length=0
temp_i=-1
for i,section in enumerate(onto_sect[:]):
    if (len(section[0])>-10):
        temp_i +=1 
        temp_sum_length+=sum_length
        print("temp_sum_length",temp_sum_length)
        section_len = section_len + len(section[0])
        word_sentence_list, seq_sentence_list,ws_logits,ws_hiddenState_all = ws(section)
        pos_sentence_list,pos_logits,pos_hiddenState,_, s_l, _,index_list = pos(word_sentence_list)
        entity_sentence_list, label_sentence_list, logits, hiddenState_all,c_v,w_v,label_list = ner(word_sentence_list, pos_sentence_list)
        print("s",sum_length)

        char_list,sectionMax,sectionMin,aver_,pie,sum_length = each_char_temp(seq_sentence_list,pos_sentence_list[0],logits,section[0],entity_sentence_list,temp_sum_length,temp_i)
        #
        temp_={}
        temp_['heatmap']=char_list
        temp_['set'] = legenSet(char_list)
        temp.append(temp_)

temp_sum_length 0
s 0
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 

s 202
1207 0
1207 1
1207 2
1207 3
1207 4
1207 5
1207 6
1207 7
1207 8
1207 9
1207 10
1207 11
1207 12
1207 13
1207 14
1207 15
1207 16
1207 17
1207 18
1207 19
1207 20
1207 21
1207 22
1207 23
1207 24
1207 25
1207 26
1207 27
1207 28
1207 29
1207 30
1207 31
1207 32
1207 33
1207 34
1207 35
1207 36
1207 37
1207 38
1207 39
1207 40
1207 41
1207 42
1207 43
1207 44
1207 45
1207 46
1207 47
1207 48
1207 49
1207 50
1207 51
1207 52
1207 53
1207 54
1207 55
1207 56
1207 57
1207 58
1207 59
1207 60
1207 61
1207 62
1207 63
1207 64
1207 65
1207 66
1207 67
1207 68
1207 69
1207 70
1207 71
1207 72
1207 73
1207 74
1207 75
1207 76
1207 77
1207 78
1207 79
1207 80
1207 81
1207 82
1207 83
1207 84
1207 85
1207 86
1207 87
1207 88
1207 89
1207 90
1207 91
1207 92
1207 93
1207 94
1207 95
1207 96
1207 97
1207 98
1207 99
1207 100
1207 101
1207 102
1207 103
1207 104
{'O': 85, 'ORG': 12, 'MONEY': 5, 'CARDINAL': 1, 'NORP': 2}
temp_sum_length 1312
s 105
1312 0
1312 1
1312 2
1312 3
1312 4
1312 5
1312 6
1312 7
1312 8
1312 9
131

s 183
2304 0
2304 1
2304 2
2304 3
2304 4
2304 5
2304 6
2304 7
2304 8
2304 9
2304 10
2304 11
2304 12
2304 13
2304 14
2304 15
2304 16
2304 17
2304 18
2304 19
2304 20
2304 21
2304 22
2304 23
2304 24
2304 25
2304 26
2304 27
2304 28
2304 29
2304 30
2304 31
2304 32
2304 33
2304 34
2304 35
2304 36
2304 37
2304 38
2304 39
2304 40
2304 41
2304 42
2304 43
2304 44
2304 45
2304 46
2304 47
2304 48
2304 49
2304 50
2304 51
2304 52
2304 53
2304 54
2304 55
2304 56
2304 57
2304 58
2304 59
2304 60
2304 61
2304 62
2304 63
2304 64
2304 65
2304 66
2304 67
2304 68
2304 69
2304 70
2304 71
2304 72
2304 73
2304 74
2304 75
2304 76
2304 77
2304 78
2304 79
2304 80
2304 81
2304 82
2304 83
2304 84
2304 85
2304 86
2304 87
2304 88
2304 89
2304 90
2304 91
2304 92
2304 93
2304 94
2304 95
2304 96
2304 97
2304 98
2304 99
2304 100
2304 101
2304 102
2304 103
2304 104
2304 105
2304 106
{'ORG': 11, 'O': 81, 'ORDINAL': 4, 'DATE': 2, 'NORP': 2, 'GPE': 7}
temp_sum_length 2411
s 107
2411 0
2411 1
2411 2
2411 3
2411 4
2411 5
2411 

s 87
3285 0
3285 1
3285 2
3285 3
3285 4
3285 5
3285 6
3285 7
3285 8
3285 9
3285 10
3285 11
3285 12
3285 13
3285 14
3285 15
3285 16
3285 17
3285 18
3285 19
3285 20
3285 21
3285 22
3285 23
3285 24
3285 25
3285 26
3285 27
3285 28
3285 29
3285 30
3285 31
3285 32
3285 33
3285 34
3285 35
3285 36
3285 37
3285 38
3285 39
3285 40
3285 41
3285 42
3285 43
3285 44
3285 45
3285 46
3285 47
3285 48
3285 49
3285 50
3285 51
3285 52
3285 53
3285 54
3285 55
3285 56
3285 57
3285 58
3285 59
3285 60
3285 61
3285 62
3285 63
3285 64
3285 65
3285 66
3285 67
3285 68
3285 69
3285 70
3285 71
3285 72
3285 73
3285 74
3285 75
3285 76
3285 77
3285 78
3285 79
3285 80
3285 81
3285 82
3285 83
3285 84
3285 85
3285 86
3285 87
3285 88
3285 89
3285 90
3285 91
3285 92
3285 93
3285 94
3285 95
3285 96
3285 97
3285 98
3285 99
3285 100
3285 101
3285 102
3285 103
3285 104
3285 105
3285 106
3285 107
3285 108
3285 109
3285 110
3285 111
3285 112
3285 113
3285 114
{'O': 85, 'GPE': 4, 'PRODUCT': 9, 'DATE': 9, 'PERCENT': 2, 'QUANTITY':

s 331
4208 0
4208 1
4208 2
4208 3
4208 4
4208 5
4208 6
4208 7
4208 8
4208 9
4208 10
4208 11
4208 12
4208 13
4208 14
4208 15
4208 16
4208 17
4208 18
4208 19
4208 20
4208 21
4208 22
4208 23
4208 24
4208 25
4208 26
4208 27
4208 28
4208 29
4208 30
4208 31
4208 32
4208 33
4208 34
4208 35
4208 36
4208 37
4208 38
4208 39
4208 40
4208 41
4208 42
4208 43
4208 44
4208 45
4208 46
4208 47
4208 48
4208 49
4208 50
4208 51
4208 52
4208 53
4208 54
4208 55
4208 56
4208 57
4208 58
4208 59
4208 60
4208 61
4208 62
4208 63
4208 64
4208 65
4208 66
4208 67
4208 68
4208 69
4208 70
4208 71
4208 72
4208 73
4208 74
4208 75
4208 76
4208 77
4208 78
4208 79
4208 80
4208 81
4208 82
4208 83
4208 84
4208 85
4208 86
4208 87
4208 88
4208 89
4208 90
4208 91
4208 92
4208 93
4208 94
4208 95
4208 96
4208 97
4208 98
4208 99
4208 100
4208 101
4208 102
4208 103
4208 104
4208 105
4208 106
4208 107
4208 108
4208 109
4208 110
4208 111
4208 112
4208 113
4208 114
4208 115
4208 116
4208 117
4208 118
{'O': 117, 'ORG': 2}
temp_sum_len

...

s 152
11305 0
11305 1
11305 2
11305 3
11305 4
11305 5
11305 6
11305 7
11305 8
11305 9
11305 10
11305 11
11305 12
11305 13
11305 14
11305 15
11305 16
11305 17
11305 18
11305 19
11305 20
11305 21
11305 22
11305 23
11305 24
11305 25
11305 26
11305 27
11305 28
11305 29
11305 30
11305 31
11305 32
11305 33
11305 34
11305 35
11305 36
11305 37
11305 38
11305 39
11305 40
11305 41
11305 42
11305 43
11305 44
11305 45
11305 46
11305 47
11305 48
11305 49
11305 50
11305 51
11305 52
11305 53
11305 54
11305 55
11305 56
11305 57
11305 58
11305 59
11305 60
11305 61
11305 62
11305 63
11305 64
11305 65
11305 66
11305 67
11305 68
11305 69
11305 70
11305 71
11305 72
11305 73
11305 74
11305 75
11305 76
11305 77
11305 78
11305 79
11305 80
11305 81
11305 82
11305 83
11305 84
11305 85
11305 86
11305 87
11305 88
11305 89
11305 90
11305 91
11305 92
11305 93
11305 94
11305 95
11305 96
11305 97
11305 98
11305 99
11305 100
11305 101
11305 102
11305 103
11305 104
11305 105
11305 106
11305 107
11305 108
11305 109
1130

s 191
12146 0
12146 1
12146 2
12146 3
12146 4
12146 5
12146 6
12146 7
12146 8
12146 9
12146 10
12146 11
12146 12
12146 13
12146 14
12146 15
12146 16
12146 17
12146 18
12146 19
12146 20
12146 21
12146 22
12146 23
12146 24
12146 25
12146 26
12146 27
12146 28
12146 29
12146 30
12146 31
12146 32
12146 33
12146 34
12146 35
12146 36
12146 37
12146 38
12146 39
12146 40
12146 41
12146 42
12146 43
12146 44
12146 45
12146 46
12146 47
12146 48
12146 49
12146 50
12146 51
12146 52
12146 53
12146 54
12146 55
12146 56
12146 57
12146 58
12146 59
12146 60
12146 61
12146 62
12146 63
12146 64
12146 65
12146 66
12146 67
12146 68
12146 69
12146 70
12146 71
12146 72
12146 73
12146 74
12146 75
12146 76
12146 77
12146 78
12146 79
12146 80
12146 81
12146 82
12146 83
12146 84
12146 85
12146 86
12146 87
12146 88
12146 89
12146 90
12146 91
12146 92
12146 93
12146 94
12146 95
12146 96
12146 97
12146 98
12146 99
12146 100
12146 101
12146 102
12146 103
12146 104
12146 105
12146 106
12146 107
12146 108
12146 109
1214

s 106
13023 0
13023 1
13023 2
13023 3
13023 4
13023 5
13023 6
13023 7
13023 8
13023 9
13023 10
13023 11
13023 12
13023 13
13023 14
13023 15
13023 16
13023 17
13023 18
13023 19
13023 20
13023 21
13023 22
13023 23
13023 24
13023 25
13023 26
13023 27
13023 28
13023 29
13023 30
13023 31
13023 32
13023 33
13023 34
13023 35
13023 36
13023 37
13023 38
13023 39
13023 40
13023 41
13023 42
13023 43
13023 44
13023 45
13023 46
13023 47
13023 48
13023 49
13023 50
13023 51
13023 52
13023 53
13023 54
13023 55
13023 56
13023 57
13023 58
13023 59
13023 60
13023 61
13023 62
13023 63
13023 64
13023 65
13023 66
13023 67
13023 68
13023 69
13023 70
13023 71
13023 72
13023 73
13023 74
13023 75
13023 76
13023 77
13023 78
13023 79
13023 80
13023 81
13023 82
13023 83
13023 84
13023 85
13023 86
13023 87
13023 88
13023 89
13023 90
13023 91
13023 92
13023 93
13023 94
13023 95
{'PERSON': 3, 'O': 90, 'ORG': 3}
temp_sum_length 13119
s 96
13119 0
13119 1
13119 2
13119 3
13119 4
13119 5
13119 6
13119 7
13119 8
13119 9


s 96
13883 0
13883 1
13883 2
13883 3
13883 4
13883 5
13883 6
13883 7
13883 8
13883 9
13883 10
13883 11
13883 12
13883 13
13883 14
13883 15
13883 16
13883 17
13883 18
13883 19
13883 20
13883 21
13883 22
13883 23
13883 24
13883 25
13883 26
13883 27
13883 28
13883 29
13883 30
13883 31
13883 32
13883 33
13883 34
13883 35
13883 36
13883 37
13883 38
13883 39
13883 40
13883 41
13883 42
13883 43
13883 44
13883 45
13883 46
13883 47
13883 48
13883 49
13883 50
13883 51
13883 52
13883 53
13883 54
13883 55
13883 56
13883 57
13883 58
13883 59
13883 60
13883 61
13883 62
13883 63
13883 64
13883 65
13883 66
13883 67
13883 68
13883 69
13883 70
13883 71
13883 72
13883 73
13883 74
13883 75
13883 76
13883 77
13883 78
13883 79
13883 80
13883 81
13883 82
13883 83
13883 84
13883 85
13883 86
13883 87
13883 88
13883 89
13883 90
13883 91
13883 92
13883 93
13883 94
13883 95
13883 96
13883 97
13883 98
13883 99
13883 100
13883 101
13883 102
13883 103
13883 104
13883 105
13883 106
13883 107
13883 108
13883 109
13883

s 285
14947 0
14947 1
14947 2
14947 3
14947 4
14947 5
14947 6
14947 7
14947 8
14947 9
14947 10
14947 11
14947 12
14947 13
14947 14
14947 15
14947 16
14947 17
14947 18
14947 19
14947 20
14947 21
14947 22
14947 23
14947 24
14947 25
14947 26
14947 27
14947 28
14947 29
14947 30
14947 31
14947 32
14947 33
14947 34
14947 35
14947 36
14947 37
14947 38
14947 39
14947 40
14947 41
14947 42
14947 43
14947 44
14947 45
14947 46
14947 47
14947 48
14947 49
14947 50
14947 51
14947 52
14947 53
14947 54
14947 55
14947 56
14947 57
14947 58
14947 59
14947 60
14947 61
14947 62
14947 63
14947 64
14947 65
14947 66
14947 67
14947 68
14947 69
14947 70
{'O': 53, 'GPE': 12, 'PERSON': 2, 'ORDINAL': 2, 'DATE': 2}
temp_sum_length 15018
s 71
15018 0
15018 1
15018 2
15018 3
15018 4
15018 5
15018 6
15018 7
15018 8
15018 9
15018 10
15018 11
15018 12
15018 13
15018 14
15018 15
15018 16
15018 17
15018 18
15018 19
15018 20
15018 21
15018 22
15018 23
15018 24
15018 25
15018 26
15018 27
15018 28
15018 29
15018 30
15018 31
1

In [40]:
char_list[0]['ner']

'O'

In [41]:
with open('news.json', 'w') as outfile:
    json.dump(temp, outfile)

In [42]:
temp_['set']

['O', 'PERSON', 'WORK_OF_ART', 'ORDINAL', 'GPE']

In [43]:
temp

[{'heatmap': [{'character': '關',
    'entropy': '0.0',
    'ner': 'O',
    'index': {'char': 0, 'article': 0}},
   {'character': '於',
    'entropy': '0.0',
    'ner': 'O',
    'index': {'char': 1, 'article': 0}},
   {'character': '「',
    'entropy': '0.0',
    'ner': 'O',
    'index': {'char': 2, 'article': 0}},
   {'character': '台',
    'entropy': '0.83',
    'ner': 'O',
    'index': {'char': 3, 'article': 0}},
   {'character': '灣',
    'entropy': '0.66',
    'ner': 'O',
    'index': {'char': 4, 'article': 0}},
   {'character': '為',
    'entropy': '0.0',
    'ner': 'O',
    'index': {'char': 5, 'article': 0}},
   {'character': '什',
    'entropy': '0.0',
    'ner': 'O',
    'index': {'char': 6, 'article': 0}},
   {'character': '麼',
    'entropy': '0.0',
    'ner': 'O',
    'index': {'char': 7, 'article': 0}},
   {'character': '沒',
    'entropy': '0.0',
    'ner': 'O',
    'index': {'char': 8, 'article': 0}},
   {'character': '有',
    'entropy': '0.0',
    'ner': 'O',
    'index': {'cha